**Group Name:** \_\_\_\_\_

**Members:** \_\_\_\_\_

## Template of the Test Code
The test code template is shown below:

In [14]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"


In [15]:
# ===============================
# Test Dataset
# ===============================
class TestDataset(Dataset):
    def __init__(self, test_dir, csv_path):
        """
        test_dir: path to test images folder
        csv_path: path to CSV file containing 'ID' and 'label' columns
        """
        self.data = pd.read_csv(csv_path, dtype={'ID': str})
        self.test_dir = test_dir
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(size=(224, 224)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_id = str(self.data.iloc[idx]['ID'])
        label = int(self.data.iloc[idx]['label'])

        img_path = os.path.join(self.test_dir, img_id + ".jpg")
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"File not found: {img_path}")

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, label




In [16]:
# ===============================
# CNN Model
# ===============================
import timm

class CNN(nn.Module):  # ← Keep name CNN() so TA's code doesn't break!
    def __init__(self, pretrained=True, freeze_backbone=True, dropout=0.3):
        super(CNN, self).__init__()

        # ViT-Base
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=pretrained, num_classes=0)  # 768-dim
        # Swin-Base
        self.swin = timm.create_model('swin_base_patch4_window7_224', pretrained=pretrained, num_classes=0)  # 1024-dim

        # Freeze backbones (highly recommended for AIGC detection)
        if freeze_backbone:
            for p in self.vit.parameters():
                p.requires_grad = False
            for p in self.swin.parameters():
                p.requires_grad = False

        # Fusion head
        self.fusion = nn.Sequential(
            nn.Linear(768 + 1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(128, 2)  # ← 2 classes: real vs AIGC
        )

    def forward(self, x):
        vit_feat = self.vit(x)                    # [B, 768]
        swin_feat = self.swin(x)                  # [B, 1024] or [B, N, 1024]

        if swin_feat.dim() == 3:  # Some timm versions return [B, N, C]
            swin_feat = swin_feat.mean(1)

        combined = torch.cat([vit_feat, swin_feat], dim=1)  # [B, 1792]
        out = self.fusion(combined)
        return out

In [17]:
# ===============================
# Test Function
# ===============================
def test_model():
    data_root = "./"
    model_path = "model.pth"

    test_dir = os.path.join(data_root, "test")
    csv_path = "submission_test_v2.csv"

    print(f"Loading test images from: {test_dir}")
    print(f"Loading labels from: {csv_path}")
    print(f"Loading model from: {model_path}")

    test_dataset = TestDataset(test_dir, csv_path)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

    model = CNN(pretrained=False, freeze_backbone=True).to(device)

    # This will now work perfectly!
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    print("Model loaded successfully!")

    model.eval()

    all_labels, all_preds = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc="Testing"):
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item()
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    final_score = 0.3 * precision + 0.3 * recall + 0.4 * f1

    print("\n" + "="*40)
    print("       TEST RESULTS (ViT-B + Swin)")
    print("="*40)
    print(f"Accuracy   : {acc:.4f}")
    print(f"Precision  : {precision:.4f}")
    print(f"Recall     : {recall:.4f}")
    print(f"F1-score   : {f1:.4f}")
    print(f"FINAL SCORE: {final_score:.4f}")
    print("="*40)


if __name__ == "__main__":
    test_model()

Loading test images from: ./test
Loading labels from: submission_test_v2.csv
Loading model from: model.pth


/var/folders/j_/v7yb81ws07d2vjpxh5tjlzqm0000gn/T/ipykernel_43348/4034885913.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_loca

Model loaded successfully!


Testing: 100%|██████████| 79/79 [03:52<00:00,  2.94s/it]


       TEST RESULTS (ViT-B + Swin)
Accuracy   : 0.4872
Precision  : 0.4851
Recall     : 0.7688
F1-score   : 0.5948
FINAL SCORE: 0.6141


## What TAs do
In this way, TAs can get the test result by only replacing the "data_root" and "model_path".

In [18]:
data_root = "path_to_dataset"
model_path = "saved_model.pth"